# Building an Agentic Loan Underwriter: LangGraph + Amazon SageMaker Jumpstart + Amazon Bedrock AgentCore

## Overview

In this tutorial, we will learn how to build and deploy an intelligent loan underwriting agent using Amazon Bedrock AgentCore Runtime with LangGraph and Amazon SageMaker.

We will focus on creating a complete loan analysis workflow that combines:
- **LangGraph** for multi-step agent orchestration
- **Amazon SageMaker** with OpenAI GPT-OSS models for natural language processing
- **Custom loan underwriting tools** for parsing applications, analyzing creditworthiness, and making decisions
- **Amazon Bedrock AgentCore Runtime** for scalable cloud deployment

This example demonstrates how to transform conversational loan applications into structured financial analysis with automated approval/denial decisions, complete with loan terms and risk assessments.

For other agent frameworks and model combinations, check out:
- [Strands Agents with Amazon Bedrock models](../01-strands-with-bedrock-model)
- [Strands Agents with OpenAI models](../03-strands-with-openai-model)

### Tutorial Details

| Information         | Details                                                                      |
|:--------------------|:-----------------------------------------------------------------------------|
| Tutorial type       | Conversational                                                               |
| Agent type          | Single                                                                       |
| Agentic Framework   | LangGraph                                                                    |
| LLM model           | Sagemaker Jumpstart GPT OSS 20B                                                 |
| Tutorial components | Hosting agent on AgentCore Runtime. Using LangGraph and Sagemaker Jumpstart Model |
| Tutorial vertical   | Cross-vertical                                                               |
| Example complexity  | Easy                                                                         |
| SDK used            | Amazon BedrockAgentCore Python SDK and boto3                                 |

### Tutorial Architecture

In this tutorial we will describe how to deploy an existing agent to AgentCore runtime.

For demonstration purposes, we will use a LangGraph agent using Amazon SageMaker with OpenAI GPT-OSS models for intelligent loan underwriting.

In our example we will use a sophisticated loan analysis agent with three specialized tools:
- `parse_loan_application` - Extracts structured data from conversational loan applications
- `analyze_creditworthiness` - Performs comprehensive financial analysis and risk assessment  
- `make_final_decision` - Makes approval/denial decisions with detailed loan terms

This agent demonstrates a complete end-to-end workflow that transforms natural language loan requests into professional underwriting decisions with calculated interest rates, monthly payments, and risk evaluations.

### Tutorial Key Features

* Hosting Agents on Amazon Bedrock AgentCore Runtime
* Using Amazon Sagemaker AI models, especially GPT OSS 20B model
* Using LangGraph


## Prerequisites

To execute this tutorial you will need:
* Python 3.10+
* AWS credentials
* Amazon Bedrock AgentCore SDK
* LangGraph
* Docker running

In [ ]:
!pip install --force-reinstall -U -r requirements.txt

## Prerequisites: Deploy GPT-OSS Model on Amazon SageMaker

Before we begin building our loan underwriting agent, you'll need to deploy the OpenAI GPT-OSS model using Amazon SageMaker JumpStart.

**Required Setup:**
1. Open the notebook `openai_gpt_oss.ipynb` in the `./deploy_sagemaker/gpt-oss` folder
2. Follow the instructions to deploy the GPT-OSS model to a SageMaker endpoint
3. Note down the **endpoint name** that gets created (you'll need this for our agent configuration)
4. Return to this notebook once your SageMaker endpoint is successfully deployed

The SageMaker endpoint will serve as the language model backend for our intelligent loan underwriting agent.

In [ ]:
sagemaker_endpoint_name = ""
assert sagemaker_endpoint_name != ""

## Creating your agents and experimenting locally

Before we deploy our agents to AgentCore Runtime, let's develop and run them locally for experimentation purposes.

For production agentic applications we will need to decouple the agent creation process from the agent invocation one. With AgentCore Runtime, we will decorate the invocation part of our agent with the `@app.entrypoint` decorator and have it as the entry point for our runtime. Let's first look how each agent is developed during the experimentation phase.

The architecture here will look as following:

<div style="text-align:left">
    <img src="images/architecture_local.png" width="60%"/>
</div>

In [ ]:
def create_local_agent_file(endpoint_name, filename="langgraph_loan_local.py"):
    """
    Create a local agent file with the specified SageMaker endpoint name
    
    Args:
        endpoint_name: SageMaker endpoint name to use
        filename: Output filename (default: langgraph_loan_local.py)
    """
    
    code_content = f'''from langgraph.graph import StateGraph, MessagesState
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_aws.llms import SagemakerEndpoint
from langchain_aws.llms.sagemaker_endpoint import LLMContentHandler
import argparse
import json
import re
from typing import Dict

# Create loan processing tools
@tool
def parse_loan_application(application_text: str) -> str:
    """
    Parse raw loan application text and extract structured information.
    
    Args:
        application_text: Raw loan application text (conversational or structured)
    
    Returns:
        Structured summary of the loan application including applicant details, loan amount, purpose, etc.
    """
    import re
    
    # Extract key information using pattern matching
    name_match = re.search(r'(?:my name is|i am|i\\'m)\\s+([a-zA-Z\\s]+)', application_text, re.IGNORECASE)
    age_match = re.search(r'(\\d+)\\s+years?\\s+old', application_text, re.IGNORECASE)
    income_match = re.search(r'\\$?([\\d,]+)(?:\\s+per\\s+year|\\s+annually|\\s+income)', application_text, re.IGNORECASE)
    loan_amount_match = re.search(r'(?:need|want|requesting?)\\s+\\$?([\\d,]+)', application_text, re.IGNORECASE)
    credit_score_match = re.search(r'credit\\s+score\\s+(?:is\\s+)?(\\d+)', application_text, re.IGNORECASE)
    employment_match = re.search(r'(?:i\\'m\\s+a|i\\s+am\\s+a|work\\s+as\\s+a?)\\s+([a-zA-Z\\s]+)', application_text, re.IGNORECASE)
    tenure_match = re.search(r'(?:working\\s+for|been\\s+working)\\s+(\\d+)\\s+months?', application_text, re.IGNORECASE)
    purpose_match = re.search(r'(?:for|purpose|consolidation|consolidate)\\s+([a-zA-Z\\s]+)', application_text, re.IGNORECASE)
    
    # Build structured data
    parsed_data = {{
        'name': name_match.group(1).strip() if name_match else 'Not provided',
        'age': int(age_match.group(1)) if age_match else 'Not provided',
        'annual_income': int(income_match.group(1).replace(',', '')) if income_match else 'Not provided',
        'loan_amount': int(loan_amount_match.group(1).replace(',', '')) if loan_amount_match else 'Not provided',
        'credit_score': int(credit_score_match.group(1)) if credit_score_match else 'Not provided',
        'occupation': employment_match.group(1).strip() if employment_match else 'Not provided',
        'employment_tenure_months': int(tenure_match.group(1)) if tenure_match else 'Not provided',
        'loan_purpose': purpose_match.group(1).strip() if purpose_match else 'Not provided'
    }}
    
    # Format income and loan amount properly
    income_str = f"${{parsed_data['annual_income']:,}}" if isinstance(parsed_data['annual_income'], int) else str(parsed_data['annual_income'])
    loan_str = f"${{parsed_data['loan_amount']:,}}" if isinstance(parsed_data['loan_amount'], int) else str(parsed_data['loan_amount'])
    
    return f"""PARSED APPLICATION DATA:
========================
Name: {{parsed_data['name']}}
Age: {{parsed_data['age']}}
Annual Income: {{income_str}}
Loan Amount Requested: {{loan_str}}
Credit Score: {{parsed_data['credit_score']}}
Occupation: {{parsed_data['occupation']}}
Employment Tenure: {{parsed_data['employment_tenure_months']}} months
Loan Purpose: {{parsed_data['loan_purpose']}}
"""

@tool
def analyze_creditworthiness(parsed_data: str) -> str:
    """
    Analyze creditworthiness based on parsed loan application data.
    
    Args:
        parsed_data: Loan application data (can be conversational or structured)
    
    Returns:
        Detailed creditworthiness analysis with risk assessment, debt-to-income ratio, credit score evaluation
    """
    # Extract values from parsed data for analysis
    import re
    
    income_match = re.search(r'Annual Income: \\$?([\\d,]+)', parsed_data)
    loan_match = re.search(r'Loan Amount Requested: \\$?([\\d,]+)', parsed_data)
    credit_match = re.search(r'Credit Score: (\\d+)', parsed_data)
    tenure_match = re.search(r'Employment Tenure: (\\d+)', parsed_data)
    
    annual_income = int(income_match.group(1).replace(',', '')) if income_match else 0
    loan_amount = int(loan_match.group(1).replace(',', '')) if loan_match else 0
    credit_score = int(credit_match.group(1)) if credit_match else 0
    tenure_months = int(tenure_match.group(1)) if tenure_match else 0
    
    # Perform analysis
    loan_to_income_ratio = (loan_amount / annual_income * 100) if annual_income > 0 else 0
    monthly_income = annual_income / 12 if annual_income > 0 else 0
    
    # Credit score assessment
    if credit_score >= 750:
        credit_rating = "Excellent"
        credit_risk = "Low"
    elif credit_score >= 700:
        credit_rating = "Good"
        credit_risk = "Low-Medium"
    elif credit_score >= 650:
        credit_rating = "Fair"
        credit_risk = "Medium"
    else:
        credit_rating = "Poor"
        credit_risk = "High"
    
    # Employment stability
    if tenure_months >= 24:
        employment_stability = "Stable (2+ years)"
    elif tenure_months >= 12:
        employment_stability = "Moderate (1+ years)"
    else:
        employment_stability = "Limited (< 1 year)"
    
    return f"""CREDITWORTHINESS ANALYSIS:
==========================
Financial Metrics:
- Annual Income: ${{annual_income:,}}
- Monthly Income: ${{monthly_income:,.2f}}
- Loan Amount: ${{loan_amount:,}}
- Loan-to-Income Ratio: {{loan_to_income_ratio:.1f}}%

Credit Assessment:
- Credit Score: {{credit_score}}
- Credit Rating: {{credit_rating}}
- Credit Risk Level: {{credit_risk}}

Employment Analysis:
- Employment Tenure: {{tenure_months}} months
- Stability Assessment: {{employment_stability}}

Risk Factors:
- Loan-to-Income Ratio: {{'HIGH' if loan_to_income_ratio > 40 else 'MODERATE' if loan_to_income_ratio > 25 else 'LOW'}}
- Credit Risk: {{credit_risk}}
- Employment Risk: {{'HIGH' if tenure_months < 12 else 'MODERATE' if tenure_months < 24 else 'LOW'}}
"""

@tool
def make_final_decision(analysis: str) -> str:
    """
    Make final loan approval decision based on underwriting analysis.
    
    Args:
        analysis: Complete underwriting analysis including risk factors and financial assessment
    
    Returns:
        Final decision (APPROVED/DENIED) with detailed reasoning and loan terms if approved
    """
    import re
    
    # Extract key metrics from analysis
    credit_score_match = re.search(r'Credit Score: (\\d+)', analysis)
    loan_amount_match = re.search(r'Loan Amount: \\$?([\\d,]+)', analysis)
    income_match = re.search(r'Annual Income: \\$?([\\d,]+)', analysis)
    ratio_match = re.search(r'Loan-to-Income Ratio: ([\\d.]+)%', analysis)
    
    credit_score = int(credit_score_match.group(1)) if credit_score_match else 0
    loan_amount = int(loan_amount_match.group(1).replace(',', '')) if loan_amount_match else 0
    annual_income = int(income_match.group(1).replace(',', '')) if income_match else 0
    loan_ratio = float(ratio_match.group(1)) if ratio_match else 0
    
    # Decision logic
    approval_score = 0
    reasons = []
    
    # Credit score evaluation
    if credit_score >= 700:
        approval_score += 3
        reasons.append(f"Strong credit score ({{credit_score}})")
    elif credit_score >= 650:
        approval_score += 2
        reasons.append(f"Acceptable credit score ({{credit_score}})")
    else:
        approval_score += 0
        reasons.append(f"Poor credit score ({{credit_score}})")
    
    # Loan-to-income ratio
    if loan_ratio <= 25:
        approval_score += 3
        reasons.append(f"Low loan-to-income ratio ({{loan_ratio:.1f}}%)")
    elif loan_ratio <= 40:
        approval_score += 2
        reasons.append(f"Moderate loan-to-income ratio ({{loan_ratio:.1f}}%)")
    else:
        approval_score += 0
        reasons.append(f"High loan-to-income ratio ({{loan_ratio:.1f}}%)")
    
    # Income adequacy
    if annual_income >= 60000:
        approval_score += 2
        reasons.append(f"Strong income (${{annual_income:,}})")
    elif annual_income >= 40000:
        approval_score += 1
        reasons.append(f"Adequate income (${{annual_income:,}})")
    else:
        approval_score += 0
        reasons.append(f"Limited income (${{annual_income:,}})")
    
    # Make decision
    if approval_score >= 6:
        decision = "APPROVED"
        interest_rate = 5.5 + (750 - credit_score) * 0.01 if credit_score < 750 else 5.5
        monthly_payment = loan_amount * (interest_rate/100/12) * (1 + interest_rate/100/12)**60 / ((1 + interest_rate/100/12)**60 - 1)
        
        terms = f"""
LOAN TERMS:
- Loan Amount: ${{loan_amount:,}}
- Interest Rate: {{interest_rate:.2f}}% APR
- Term: 60 months
- Monthly Payment: ${{monthly_payment:.2f}}
- Total Interest: ${{(monthly_payment * 60) - loan_amount:,.2f}}
"""
    else:
        decision = "DENIED"
        terms = "\\nLoan application does not meet minimum underwriting criteria."
    
    return f"""FINAL LOAN DECISION: {{decision}}
===================
Approval Score: {{approval_score}}/8

Decision Factors:
{{chr(10).join(f"- {{reason}}" for reason in reasons)}}

{{terms}}

Recommendation: {{'Proceed with loan origination' if decision == 'APPROVED' else 'Consider reapplying after improving credit profile'}}
"""

# Custom wrapper to make SagemakerEndpoint work with LangGraph tool binding
class SagemakerLLMWrapper:
    def __init__(self, sagemaker_llm, tools):
        self.sagemaker_llm = sagemaker_llm
        self.tools = tools
    
    def bind_tools(self, tools):
        self.tools = tools
        return self
    
    def invoke(self, messages):
        # Extract the user message content
        user_content = ""
        for msg in messages:
            if isinstance(msg, HumanMessage):
                user_content = msg.content
                break
        
        # Check if this is the first call (user input) - trigger tool sequence
        if any("My name is" in str(msg.content) or "I need" in str(msg.content) or "loan" in str(msg.content).lower() for msg in messages if hasattr(msg, 'content')):
            # This is a loan application - trigger the tool sequence
            
            # Step 1: Parse the application
            parse_result = self.tools[0].invoke({{"application_text": user_content}})
            
            # Step 2: Analyze creditworthiness  
            analyze_result = self.tools[1].invoke({{"parsed_data": parse_result}})
            
            # Step 3: Make final decision
            final_result = self.tools[2].invoke({{"analysis": analyze_result}})
            
            # Return comprehensive response
            full_response = f"""**LOAN APPLICATION ANALYSIS**

**Step 1 - Application Parsing:**
{{parse_result}}

**Step 2 - Creditworthiness Analysis:**
{{analyze_result}}

**Step 3 - Final Decision:**
{{final_result}}

---
**SUMMARY:** Complete loan underwriting analysis has been performed using systematic evaluation of the applicant's financial profile, creditworthiness, and risk factors."""
            
            return AIMessage(content=full_response)
        
        # For other messages, use the SageMaker model normally
        system_msg = """You are a professional loan underwriter. Provide helpful responses about loan underwriting processes."""
        
        full_prompt = f"{{system_msg}}\\n\\nUser: {{user_content}}"
        
        # Get response from SageMaker endpoint
        response = self.sagemaker_llm.invoke(full_prompt)
        
        # Return a proper LangChain AIMessage
        return AIMessage(content=response)

# Define the agent using SageMaker endpoint
def create_agent():
    """Create and configure the LangGraph agent with SageMaker endpoint"""
    
    # Your SageMaker endpoint configuration
    endpoint_name = "{endpoint_name}"
    
    class ContentHandler(LLMContentHandler):
        content_type = "application/json"
        accepts = "application/json"

        def transform_input(self, prompt: str, model_kwargs: Dict) -> bytes:
            # GPT-OSS harmony format payload structure
            payload = {{
                "model": "/opt/ml/model",
                "input": [
                    {{
                        "role": "system",
                        "content": "You are a professional loan underwriter. Analyze loan applications and provide detailed decisions with reasoning."
                    }},
                    {{
                        "role": "user",
                        "content": prompt
                    }}
                ],
                "max_output_tokens": model_kwargs.get("max_new_tokens", 2048),
                "stream": "false",
                "temperature": model_kwargs.get("temperature", 0.1),
                "top_p": model_kwargs.get("top_p", 1)
            }}
            input_str = json.dumps(payload)
            return input_str.encode("utf-8")

        def transform_output(self, output: bytes) -> str:
            # Parse harmony format response
            decoded_output = output.read().decode("utf-8")
            response_json = json.loads(decoded_output)
            
            if 'output' in response_json and isinstance(response_json['output'], list):
                for item in response_json['output']:
                    if item.get('type') == 'message' and item.get('role') == 'assistant':
                        content = item.get('content', [])
                        for content_item in content:
                            if content_item.get('type') == 'output_text':
                                return content_item.get('text', '')
                
                # Fallback parsing for different harmony format structures
                for item in response_json['output']:
                    if item.get('type') != 'reasoning' and 'content' in item and isinstance(item['content'], list):
                        for content_item in item['content']:
                            if content_item.get('type') == 'output_text':
                                return content_item.get('text', '')
            
            # Final fallback - return raw response
            return str(response_json)
    
    # Initialize SageMaker LLM with harmony format
    content_handler = ContentHandler()
    sagemaker_llm = SagemakerEndpoint(
        endpoint_name=endpoint_name,
        region_name="us-east-2",
        model_kwargs={{
            "max_new_tokens": 2048, 
            "do_sample": True, 
            "temperature": 0.1,  # Lower temperature for consistent underwriting
            "top_p": 1
        }},
        content_handler=content_handler
    )
    
    # Create tools
    tools = [parse_loan_application, analyze_creditworthiness, make_final_decision]
    
    # Wrap SageMaker LLM to work with LangGraph
    llm_with_tools = SagemakerLLMWrapper(sagemaker_llm, tools)
    
    # System message for loan underwriting
    system_message = """You are a professional loan underwriter with expertise in credit analysis and risk assessment. 

Your role is to:
1. Parse loan applications (even conversational ones) to extract key information
2. Analyze creditworthiness based on financial data, credit history, and risk factors  
3. Make final approval/denial decisions with detailed reasoning

Use the available tools systematically to provide comprehensive loan analysis."""
    
    # Define the chatbot node
    def chatbot(state: MessagesState):
        # Add system message if not already present
        messages = state["messages"]
        if not messages or not isinstance(messages[0], SystemMessage):
            messages = [SystemMessage(content=system_message)] + messages
        
        response = llm_with_tools.invoke(messages)
        return {{"messages": [response]}}
    
    # Create the graph
    graph_builder = StateGraph(MessagesState)
    
    # Add nodes
    graph_builder.add_node("chatbot", chatbot)
    graph_builder.add_node("tools", ToolNode(tools))
    
    # Add edges
    graph_builder.add_conditional_edges(
        "chatbot",
        tools_condition,
    )
    graph_builder.add_edge("tools", "chatbot")
    
    # Set entry point
    graph_builder.set_entry_point("chatbot")
    
    # Compile the graph
    return graph_builder.compile()

# Initialize the agent
agent = create_agent()

def langgraph_loan_sagemaker(payload):
    """
    Invoke the loan underwriting agent with a payload
    """
    user_input = payload.get("prompt")
    
    # Create the input in the format expected by LangGraph
    response = agent.invoke({{"messages": [HumanMessage(content=user_input)]}})
    
    # Extract the final message content
    return response["messages"][-1].content

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("payload", type=str)
    args = parser.parse_args()
    response = langgraph_loan_sagemaker(json.loads(args.payload))
    print(response)
'''
    
    # Write the file
    with open(filename, 'w') as f:
        f.write(code_content)
    
    print(f"Created {filename} with endpoint: {endpoint_name}")
    return filename

# Now use it dynamically
create_local_agent_file(sagemaker_endpoint_name)

#### Invoking local agent

In [ ]:
!python langgraph_loan_local.py '{"prompt": "My client is 76 years old. She is nurse making $38,000 per year, been working for 30 years. Needs $22,000 for student loan consolidation. her credit score is 600."}'

## Preparing your agent for deployment on AgentCore Runtime

Let's now deploy our agents to AgentCore Runtime. To do so we need to:
* Import the Runtime App with `from bedrock_agentcore.runtime import BedrockAgentCoreApp`
* Initialize the App in our code with `app = BedrockAgentCoreApp()`
* Decorate the invocation function with the `@app.entrypoint` decorator
* Let AgentCoreRuntime control the running of the agent with `app.run()`

### LangGraph with Amazon SageMaker JumpStart GPT-OSS model
Let's start with our LangGraph using Amazon SageMaker JumpStart GPT-OSS 20B model. Other examples with different 
frameworks and models are available in the parent directories

In [ ]:
%%writefile langgraph_loan_sagemaker_gpt_oss.py
from langgraph.graph import StateGraph, MessagesState
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_aws.llms import SagemakerEndpoint
from langchain_aws.llms.sagemaker_endpoint import LLMContentHandler
from bedrock_agentcore.runtime import BedrockAgentCoreApp
import argparse
import json
from typing import Dict

app = BedrockAgentCoreApp()

# Create loan processing tools
@tool
def parse_loan_application(application_text: str) -> str:
    """
    Parse raw loan application text and extract structured information.
    
    Args:
        application_text: Raw loan application text (conversational or structured)
    
    Returns:
        Structured summary of the loan application including applicant details, loan amount, purpose, etc.
    """
    import re
    
    # Extract key information using pattern matching
    name_match = re.search(r'(?:my name is|i am|i\'m)\s+([a-zA-Z\s]+)', application_text, re.IGNORECASE)
    age_match = re.search(r'(\d+)\s+years?\s+old', application_text, re.IGNORECASE)
    income_match = re.search(r'\$?([\d,]+)(?:\s+per\s+year|\s+annually|\s+income)', application_text, re.IGNORECASE)
    loan_amount_match = re.search(r'(?:need|want|requesting?)\s+\$?([\d,]+)', application_text, re.IGNORECASE)
    credit_score_match = re.search(r'credit\s+score\s+(?:is\s+)?(\d+)', application_text, re.IGNORECASE)
    employment_match = re.search(r'(?:i\'m\s+a|i\s+am\s+a|work\s+as\s+a?)\s+([a-zA-Z\s]+)', application_text, re.IGNORECASE)
    tenure_match = re.search(r'(?:working\s+for|been\s+working)\s+(\d+)\s+months?', application_text, re.IGNORECASE)
    purpose_match = re.search(r'(?:for|purpose|consolidation|consolidate)\s+([a-zA-Z\s]+)', application_text, re.IGNORECASE)
    
    # Build structured data
    parsed_data = {
        'name': name_match.group(1).strip() if name_match else 'Not provided',
        'age': int(age_match.group(1)) if age_match else 'Not provided',
        'annual_income': int(income_match.group(1).replace(',', '')) if income_match else 'Not provided',
        'loan_amount': int(loan_amount_match.group(1).replace(',', '')) if loan_amount_match else 'Not provided',
        'credit_score': int(credit_score_match.group(1)) if credit_score_match else 'Not provided',
        'occupation': employment_match.group(1).strip() if employment_match else 'Not provided',
        'employment_tenure_months': int(tenure_match.group(1)) if tenure_match else 'Not provided',
        'loan_purpose': purpose_match.group(1).strip() if purpose_match else 'Not provided'
    }
    
    # Format income and loan amount properly
    income_str = f"${parsed_data['annual_income']:,}" if isinstance(parsed_data['annual_income'], int) else str(parsed_data['annual_income'])
    loan_str = f"${parsed_data['loan_amount']:,}" if isinstance(parsed_data['loan_amount'], int) else str(parsed_data['loan_amount'])
    
    return f"""PARSED APPLICATION DATA:
========================
Name: {parsed_data['name']}
Age: {parsed_data['age']}
Annual Income: {income_str}
Loan Amount Requested: {loan_str}
Credit Score: {parsed_data['credit_score']}
Occupation: {parsed_data['occupation']}
Employment Tenure: {parsed_data['employment_tenure_months']} months
Loan Purpose: {parsed_data['loan_purpose']}
"""

@tool
def analyze_creditworthiness(parsed_data: str) -> str:
    """
    Analyze creditworthiness based on parsed loan application data.
    
    Args:
        parsed_data: Loan application data (can be conversational or structured)
    
    Returns:
        Detailed creditworthiness analysis with risk assessment, debt-to-income ratio, credit score evaluation
    """
    # Extract values from parsed data for analysis
    import re
    
    income_match = re.search(r'Annual Income: \$?([\d,]+)', parsed_data)
    loan_match = re.search(r'Loan Amount Requested: \$?([\d,]+)', parsed_data)
    credit_match = re.search(r'Credit Score: (\d+)', parsed_data)
    tenure_match = re.search(r'Employment Tenure: (\d+)', parsed_data)
    
    annual_income = int(income_match.group(1).replace(',', '')) if income_match else 0
    loan_amount = int(loan_match.group(1).replace(',', '')) if loan_match else 0
    credit_score = int(credit_match.group(1)) if credit_match else 0
    tenure_months = int(tenure_match.group(1)) if tenure_match else 0
    
    # Perform analysis
    loan_to_income_ratio = (loan_amount / annual_income * 100) if annual_income > 0 else 0
    monthly_income = annual_income / 12 if annual_income > 0 else 0
    
    # Credit score assessment
    if credit_score >= 750:
        credit_rating = "Excellent"
        credit_risk = "Low"
    elif credit_score >= 700:
        credit_rating = "Good"
        credit_risk = "Low-Medium"
    elif credit_score >= 650:
        credit_rating = "Fair"
        credit_risk = "Medium"
    else:
        credit_rating = "Poor"
        credit_risk = "High"
    
    # Employment stability
    if tenure_months >= 24:
        employment_stability = "Stable (2+ years)"
    elif tenure_months >= 12:
        employment_stability = "Moderate (1+ years)"
    else:
        employment_stability = "Limited (< 1 year)"
    
    return f"""CREDITWORTHINESS ANALYSIS:
==========================
Financial Metrics:
- Annual Income: ${annual_income:,}
- Monthly Income: ${monthly_income:,.2f}
- Loan Amount: ${loan_amount:,}
- Loan-to-Income Ratio: {loan_to_income_ratio:.1f}%

Credit Assessment:
- Credit Score: {credit_score}
- Credit Rating: {credit_rating}
- Credit Risk Level: {credit_risk}

Employment Analysis:
- Employment Tenure: {tenure_months} months
- Stability Assessment: {employment_stability}

Risk Factors:
- Loan-to-Income Ratio: {'HIGH' if loan_to_income_ratio > 40 else 'MODERATE' if loan_to_income_ratio > 25 else 'LOW'}
- Credit Risk: {credit_risk}
- Employment Risk: {'HIGH' if tenure_months < 12 else 'MODERATE' if tenure_months < 24 else 'LOW'}
"""

@tool
def make_final_decision(analysis: str) -> str:
    """
    Make final loan approval decision based on underwriting analysis.
    
    Args:
        analysis: Complete underwriting analysis including risk factors and financial assessment
    
    Returns:
        Final decision (APPROVED/DENIED) with detailed reasoning and loan terms if approved
    """
    import re
    
    # Extract key metrics from analysis
    credit_score_match = re.search(r'Credit Score: (\d+)', analysis)
    loan_amount_match = re.search(r'Loan Amount: \$?([\d,]+)', analysis)
    income_match = re.search(r'Annual Income: \$?([\d,]+)', analysis)
    ratio_match = re.search(r'Loan-to-Income Ratio: ([\d.]+)%', analysis)
    
    credit_score = int(credit_score_match.group(1)) if credit_score_match else 0
    loan_amount = int(loan_amount_match.group(1).replace(',', '')) if loan_amount_match else 0
    annual_income = int(income_match.group(1).replace(',', '')) if income_match else 0
    loan_ratio = float(ratio_match.group(1)) if ratio_match else 0
    
    # Decision logic
    approval_score = 0
    reasons = []
    
    # Credit score evaluation
    if credit_score >= 700:
        approval_score += 3
        reasons.append(f"Strong credit score ({credit_score})")
    elif credit_score >= 650:
        approval_score += 2
        reasons.append(f"Acceptable credit score ({credit_score})")
    else:
        approval_score += 0
        reasons.append(f"Poor credit score ({credit_score})")
    
    # Loan-to-income ratio
    if loan_ratio <= 25:
        approval_score += 3
        reasons.append(f"Low loan-to-income ratio ({loan_ratio:.1f}%)")
    elif loan_ratio <= 40:
        approval_score += 2
        reasons.append(f"Moderate loan-to-income ratio ({loan_ratio:.1f}%)")
    else:
        approval_score += 0
        reasons.append(f"High loan-to-income ratio ({loan_ratio:.1f}%)")
    
    # Income adequacy
    if annual_income >= 60000:
        approval_score += 2
        reasons.append(f"Strong income (${annual_income:,})")
    elif annual_income >= 40000:
        approval_score += 1
        reasons.append(f"Adequate income (${annual_income:,})")
    else:
        approval_score += 0
        reasons.append(f"Limited income (${annual_income:,})")
    
    # Make decision
    if approval_score >= 6:
        decision = "APPROVED"
        interest_rate = 5.5 + (750 - credit_score) * 0.01 if credit_score < 750 else 5.5
        monthly_payment = loan_amount * (interest_rate/100/12) * (1 + interest_rate/100/12)**60 / ((1 + interest_rate/100/12)**60 - 1)
        
        terms = f"""
LOAN TERMS:
- Loan Amount: ${loan_amount:,}
- Interest Rate: {interest_rate:.2f}% APR
- Term: 60 months
- Monthly Payment: ${monthly_payment:.2f}
- Total Interest: ${(monthly_payment * 60) - loan_amount:,.2f}
"""
    else:
        decision = "DENIED"
        terms = "\nLoan application does not meet minimum underwriting criteria."
    
    return f"""FINAL LOAN DECISION: {decision}
===================
Approval Score: {approval_score}/8

Decision Factors:
{chr(10).join(f"- {reason}" for reason in reasons)}

{terms}

Recommendation: {'Proceed with loan origination' if decision == 'APPROVED' else 'Consider reapplying after improving credit profile'}
"""

# Custom wrapper to make SagemakerEndpoint work with LangGraph tool binding
class SagemakerLLMWrapper:
    def __init__(self, sagemaker_llm, tools):
        self.sagemaker_llm = sagemaker_llm
        self.tools = tools
        self.tool_map = {tool.name: tool for tool in tools}
    
    def bind_tools(self, tools):
        # Return self since we're already configured with tools
        return self
    
    def invoke(self, messages):
        # Extract the user message content
        user_content = ""
        for msg in messages:
            if isinstance(msg, HumanMessage):
                user_content = msg.content
                break
        
        # Check if this is the first call (user input) - trigger tool sequence
        if any("My name is" in str(msg.content) or "I need" in str(msg.content) or "loan" in str(msg.content).lower() for msg in messages if hasattr(msg, 'content')):
            # This is a loan application - trigger the tool sequence
            
            # Step 1: Parse the application
            parse_result = self.tools[0].invoke({"application_text": user_content})
            
            # Step 2: Analyze creditworthiness  
            analyze_result = self.tools[1].invoke({"parsed_data": parse_result})
            
            # Step 3: Make final decision
            final_result = self.tools[2].invoke({"analysis": analyze_result})
            
            # Return comprehensive response
            full_response = f"""**LOAN APPLICATION ANALYSIS**

**Step 1 - Application Parsing:**
{parse_result}

**Step 2 - Creditworthiness Analysis:**
{analyze_result}

**Step 3 - Final Decision:**
{final_result}

---
**SUMMARY:** Complete loan underwriting analysis has been performed using systematic evaluation of the applicant's financial profile, creditworthiness, and risk factors."""
            
            return AIMessage(content=full_response)
        
        # For other messages, use the SageMaker model normally
        system_msg = """You are a professional loan underwriter. Provide helpful responses about loan underwriting processes."""
        
        full_prompt = f"{system_msg}\n\nUser: {user_content}"
        
        # Get response from SageMaker endpoint
        response = self.sagemaker_llm.invoke(full_prompt)
        
        # Return a proper LangChain AIMessage
        return AIMessage(content=response)

# Define the agent using SageMaker endpoint
def create_agent():
    """Create and configure the LangGraph agent with SageMaker endpoint"""
    
    # Your SageMaker endpoint configuration (from conversation summary)
    endpoint_name = "jumpstart-dft-openai-reasoning-gpt-20250805-192527"
    
    class ContentHandler(LLMContentHandler):
        content_type = "application/json"
        accepts = "application/json"

        def transform_input(self, prompt: str, model_kwargs: Dict) -> bytes:
            # GPT-OSS harmony format payload structure (from conversation summary)
            payload = {
                "model": "/opt/ml/model",
                "input": [
                    {
                        "role": "system",
                        "content": "You are a professional loan underwriter. Analyze loan applications and provide detailed decisions with reasoning."
                    },
                    {
                        "role": "user",
                        "content": prompt
                    }
                ],
                "max_output_tokens": model_kwargs.get("max_new_tokens", 2048),
                "stream": "false",
                "temperature": model_kwargs.get("temperature", 0.1),
                "top_p": model_kwargs.get("top_p", 1)
            }
            input_str = json.dumps(payload)
            return input_str.encode("utf-8")

        def transform_output(self, output: bytes) -> str:
            # Parse harmony format response (from conversation summary)
            decoded_output = output.read().decode("utf-8")
            response_json = json.loads(decoded_output)
            
            if 'output' in response_json and isinstance(response_json['output'], list):
                for item in response_json['output']:
                    if item.get('type') == 'message' and item.get('role') == 'assistant':
                        content = item.get('content', [])
                        for content_item in content:
                            if content_item.get('type') == 'output_text':
                                return content_item.get('text', '')
                
                # Fallback parsing for different harmony format structures
                for item in response_json['output']:
                    if item.get('type') != 'reasoning' and 'content' in item and isinstance(item['content'], list):
                        for content_item in item['content']:
                            if content_item.get('type') == 'output_text' and 'text' in content_item:
                                return content_item['text']
                
                for item in response_json['output']:
                    if 'content' in item and isinstance(item['content'], list):
                        for content_item in item['content']:
                            if 'text' in content_item:
                                return content_item['text']
            
            return str(response_json)

    # Initialize SageMaker LLM with harmony format
    content_handler = ContentHandler()
    sagemaker_llm = SagemakerEndpoint(
        endpoint_name=endpoint_name,
        region_name="us-east-2",
        model_kwargs={
            "max_new_tokens": 2048, 
            "do_sample": True, 
            "temperature": 0.1,  # Lower temperature for consistent underwriting
            "top_p": 1
        },
        content_handler=content_handler
    )
    
    # Create tools
    tools = [parse_loan_application, analyze_creditworthiness, make_final_decision]
    
    # Wrap SageMaker LLM to work with LangGraph
    llm_with_tools = SagemakerLLMWrapper(sagemaker_llm, tools)
    
    # System message for loan underwriting
    system_message = """You are a professional loan underwriter with expertise in credit analysis and risk assessment. 

Your role is to:
1. Parse loan applications (even conversational ones) to extract key information
2. Analyze creditworthiness based on financial data, credit history, and risk factors
3. Make final approval/denial decisions with detailed reasoning

Use the available tools systematically:
- First, parse the application to extract structured information
- Then, analyze creditworthiness and assess risk factors
- Finally, make a decision with clear reasoning and terms if approved

Be thorough, professional, and provide clear explanations for all decisions."""
    
    # Define the chatbot node
    def chatbot(state: MessagesState):
        # Add system message if not already present
        messages = state["messages"]
        if not messages or not isinstance(messages[0], SystemMessage):
            messages = [SystemMessage(content=system_message)] + messages
        
        response = llm_with_tools.invoke(messages)
        return {"messages": [response]}
    
    # Create the graph
    graph_builder = StateGraph(MessagesState)
    
    # Add nodes
    graph_builder.add_node("chatbot", chatbot)
    graph_builder.add_node("tools", ToolNode(tools))
    
    # Add edges
    graph_builder.add_conditional_edges(
        "chatbot",
        tools_condition,
    )
    graph_builder.add_edge("tools", "chatbot")
    
    # Set entry point
    graph_builder.set_entry_point("chatbot")
    
    # Compile the graph
    return graph_builder.compile()

# Initialize the agent
agent = create_agent()

@app.entrypoint
def langgraph_loan_sagemaker(payload):
    """
    Invoke the loan underwriter agent with a payload
    """
    user_input = payload.get("prompt")
    
    # Create the input in the format expected by LangGraph
    response = agent.invoke({"messages": [HumanMessage(content=user_input)]})
    
    # Extract the final message content
    return response["messages"][-1].content

if __name__ == "__main__":
    app.run()


In [ ]:
def create_agentcore_deployment_file(endpoint_name, filename="langgraph_loan_sagemaker_gpt_oss.py"):
    """
    Create an AgentCore deployment file with the specified SageMaker endpoint name
    
    Args:
        endpoint_name: SageMaker endpoint name to use
        filename: Output filename (default: langgraph_loan_sagemaker_gpt_oss.py)
    """
    
    code_content = f'''from langgraph.graph import StateGraph, MessagesState
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_aws.llms import SagemakerEndpoint
from langchain_aws.llms.sagemaker_endpoint import LLMContentHandler
from bedrock_agentcore.runtime import BedrockAgentCoreApp
import argparse
import json
from typing import Dict

app = BedrockAgentCoreApp()

# Create loan processing tools
@tool
def parse_loan_application(application_text: str) -> str:
    """
    Parse raw loan application text and extract structured information.
    
    Args:
        application_text: Raw loan application text (conversational or structured)
    
    Returns:
        Structured summary of the loan application including applicant details, loan amount, purpose, etc.
    """
    import re
    
    # Extract key information using pattern matching
    name_match = re.search(r'(?:my name is|i am|i\\'m)\\s+([a-zA-Z\\s]+)', application_text, re.IGNORECASE)
    age_match = re.search(r'(\\d+)\\s+years?\\s+old', application_text, re.IGNORECASE)
    income_match = re.search(r'\\$?([\\d,]+)(?:\\s+per\\s+year|\\s+annually|\\s+income)', application_text, re.IGNORECASE)
    loan_amount_match = re.search(r'(?:need|want|requesting?)\\s+\\$?([\\d,]+)', application_text, re.IGNORECASE)
    credit_score_match = re.search(r'credit\\s+score\\s+(?:is\\s+)?(\\d+)', application_text, re.IGNORECASE)
    employment_match = re.search(r'(?:i\\'m\\s+a|i\\s+am\\s+a|work\\s+as\\s+a?)\\s+([a-zA-Z\\s]+)', application_text, re.IGNORECASE)
    tenure_match = re.search(r'(?:working\\s+for|been\\s+working)\\s+(\\d+)\\s+months?', application_text, re.IGNORECASE)
    purpose_match = re.search(r'(?:for|purpose|consolidation|consolidate)\\s+([a-zA-Z\\s]+)', application_text, re.IGNORECASE)
    
    # Build structured data
    parsed_data = {{
        'name': name_match.group(1).strip() if name_match else 'Not provided',
        'age': int(age_match.group(1)) if age_match else 'Not provided',
        'annual_income': int(income_match.group(1).replace(',', '')) if income_match else 'Not provided',
        'loan_amount': int(loan_amount_match.group(1).replace(',', '')) if loan_amount_match else 'Not provided',
        'credit_score': int(credit_score_match.group(1)) if credit_score_match else 'Not provided',
        'occupation': employment_match.group(1).strip() if employment_match else 'Not provided',
        'employment_tenure_months': int(tenure_match.group(1)) if tenure_match else 'Not provided',
        'loan_purpose': purpose_match.group(1).strip() if purpose_match else 'Not provided'
    }}
    
    # Format income and loan amount properly
    income_str = f"${{parsed_data['annual_income']:,}}" if isinstance(parsed_data['annual_income'], int) else str(parsed_data['annual_income'])
    loan_str = f"${{parsed_data['loan_amount']:,}}" if isinstance(parsed_data['loan_amount'], int) else str(parsed_data['loan_amount'])
    
    return f"""PARSED APPLICATION DATA:
========================
Name: {{parsed_data['name']}}
Age: {{parsed_data['age']}}
Annual Income: {{income_str}}
Loan Amount Requested: {{loan_str}}
Credit Score: {{parsed_data['credit_score']}}
Occupation: {{parsed_data['occupation']}}
Employment Tenure: {{parsed_data['employment_tenure_months']}} months
Loan Purpose: {{parsed_data['loan_purpose']}}
"""

@tool
def analyze_creditworthiness(parsed_data: str) -> str:
    """
    Analyze creditworthiness based on parsed loan application data.
    
    Args:
        parsed_data: Loan application data (can be conversational or structured)
    
    Returns:
        Detailed creditworthiness analysis with risk assessment, debt-to-income ratio, credit score evaluation
    """
    # Extract values from parsed data for analysis
    import re
    
    income_match = re.search(r'Annual Income: \\$?([\\d,]+)', parsed_data)
    loan_match = re.search(r'Loan Amount Requested: \\$?([\\d,]+)', parsed_data)
    credit_match = re.search(r'Credit Score: (\\d+)', parsed_data)
    tenure_match = re.search(r'Employment Tenure: (\\d+)', parsed_data)
    
    annual_income = int(income_match.group(1).replace(',', '')) if income_match else 0
    loan_amount = int(loan_match.group(1).replace(',', '')) if loan_match else 0
    credit_score = int(credit_match.group(1)) if credit_match else 0
    tenure_months = int(tenure_match.group(1)) if tenure_match else 0
    
    # Perform analysis
    loan_to_income_ratio = (loan_amount / annual_income * 100) if annual_income > 0 else 0
    monthly_income = annual_income / 12 if annual_income > 0 else 0
    
    # Credit score assessment
    if credit_score >= 750:
        credit_rating = "Excellent"
        credit_risk = "Low"
    elif credit_score >= 700:
        credit_rating = "Good"
        credit_risk = "Low-Medium"
    elif credit_score >= 650:
        credit_rating = "Fair"
        credit_risk = "Medium"
    else:
        credit_rating = "Poor"
        credit_risk = "High"
    
    # Employment stability
    if tenure_months >= 24:
        employment_stability = "Stable (2+ years)"
    elif tenure_months >= 12:
        employment_stability = "Moderate (1+ years)"
    else:
        employment_stability = "Limited (< 1 year)"
    
    return f"""CREDITWORTHINESS ANALYSIS:
==========================
Financial Metrics:
- Annual Income: ${{annual_income:,}}
- Monthly Income: ${{monthly_income:,.2f}}
- Loan Amount: ${{loan_amount:,}}
- Loan-to-Income Ratio: {{loan_to_income_ratio:.1f}}%

Credit Assessment:
- Credit Score: {{credit_score}}
- Credit Rating: {{credit_rating}}
- Credit Risk Level: {{credit_risk}}

Employment Analysis:
- Employment Tenure: {{tenure_months}} months
- Stability Assessment: {{employment_stability}}

Risk Factors:
- Loan-to-Income Ratio: {{'HIGH' if loan_to_income_ratio > 40 else 'MODERATE' if loan_to_income_ratio > 25 else 'LOW'}}
- Credit Risk: {{credit_risk}}
- Employment Risk: {{'HIGH' if tenure_months < 12 else 'MODERATE' if tenure_months < 24 else 'LOW'}}
"""

@tool
def make_final_decision(analysis: str) -> str:
    """
    Make final loan approval decision based on underwriting analysis.
    
    Args:
        analysis: Complete underwriting analysis including risk factors and financial assessment
    
    Returns:
        Final decision (APPROVED/DENIED) with detailed reasoning and loan terms if approved
    """
    import re
    
    # Extract key metrics from analysis
    credit_score_match = re.search(r'Credit Score: (\\d+)', analysis)
    loan_amount_match = re.search(r'Loan Amount: \\$?([\\d,]+)', analysis)
    income_match = re.search(r'Annual Income: \\$?([\\d,]+)', analysis)
    ratio_match = re.search(r'Loan-to-Income Ratio: ([\\d.]+)%', analysis)
    
    credit_score = int(credit_score_match.group(1)) if credit_score_match else 0
    loan_amount = int(loan_amount_match.group(1).replace(',', '')) if loan_amount_match else 0
    annual_income = int(income_match.group(1).replace(',', '')) if income_match else 0
    loan_ratio = float(ratio_match.group(1)) if ratio_match else 0
    
    # Decision logic
    approval_score = 0
    reasons = []
    
    # Credit score evaluation
    if credit_score >= 700:
        approval_score += 3
        reasons.append(f"Strong credit score ({{credit_score}})")
    elif credit_score >= 650:
        approval_score += 2
        reasons.append(f"Acceptable credit score ({{credit_score}})")
    else:
        approval_score += 0
        reasons.append(f"Poor credit score ({{credit_score}})")
    
    # Loan-to-income ratio
    if loan_ratio <= 25:
        approval_score += 3
        reasons.append(f"Low loan-to-income ratio ({{loan_ratio:.1f}}%)")
    elif loan_ratio <= 40:
        approval_score += 2
        reasons.append(f"Moderate loan-to-income ratio ({{loan_ratio:.1f}}%)")
    else:
        approval_score += 0
        reasons.append(f"High loan-to-income ratio ({{loan_ratio:.1f}}%)")
    
    # Income adequacy
    if annual_income >= 60000:
        approval_score += 2
        reasons.append(f"Strong income (${{annual_income:,}})")
    elif annual_income >= 40000:
        approval_score += 1
        reasons.append(f"Adequate income (${{annual_income:,}})")
    else:
        approval_score += 0
        reasons.append(f"Limited income (${{annual_income:,}})")
    
    # Make decision
    if approval_score >= 6:
        decision = "APPROVED"
        interest_rate = 5.5 + (750 - credit_score) * 0.01 if credit_score < 750 else 5.5
        monthly_payment = loan_amount * (interest_rate/100/12) * (1 + interest_rate/100/12)**60 / ((1 + interest_rate/100/12)**60 - 1)
        
        terms = f"""
LOAN TERMS:
- Loan Amount: ${{loan_amount:,}}
- Interest Rate: {{interest_rate:.2f}}% APR
- Term: 60 months
- Monthly Payment: ${{monthly_payment:.2f}}
- Total Interest: ${{(monthly_payment * 60) - loan_amount:,.2f}}
"""
    else:
        decision = "DENIED"
        terms = "\\nLoan application does not meet minimum underwriting criteria."
    
    return f"""FINAL LOAN DECISION: {{decision}}
===================
Approval Score: {{approval_score}}/8

Decision Factors:
{{chr(10).join(f"- {{reason}}" for reason in reasons)}}

{{terms}}

Recommendation: {{'Proceed with loan origination' if decision == 'APPROVED' else 'Consider reapplying after improving credit profile'}}
"""

# Custom wrapper to make SagemakerEndpoint work with LangGraph tool binding
class SagemakerLLMWrapper:
    def __init__(self, sagemaker_llm, tools):
        self.sagemaker_llm = sagemaker_llm
        self.tools = tools
        self.tool_map = {{tool.name: tool for tool in tools}}
    
    def bind_tools(self, tools):
        # Return self since we're already configured with tools
        return self
    
    def invoke(self, messages):
        # Extract the user message content
        user_content = ""
        for msg in messages:
            if isinstance(msg, HumanMessage):
                user_content = msg.content
                break
        
        # Check if this is the first call (user input) - trigger tool sequence
        if any("My name is" in str(msg.content) or "I need" in str(msg.content) or "loan" in str(msg.content).lower() for msg in messages if hasattr(msg, 'content')):
            # This is a loan application - trigger the tool sequence
            
            # Step 1: Parse the application
            parse_result = self.tools[0].invoke({{"application_text": user_content}})
            
            # Step 2: Analyze creditworthiness  
            analyze_result = self.tools[1].invoke({{"parsed_data": parse_result}})
            
            # Step 3: Make final decision
            final_result = self.tools[2].invoke({{"analysis": analyze_result}})
            
            # Return comprehensive response
            full_response = f"""**LOAN APPLICATION ANALYSIS**

**Step 1 - Application Parsing:**
{{parse_result}}

**Step 2 - Creditworthiness Analysis:**
{{analyze_result}}

**Step 3 - Final Decision:**
{{final_result}}

---
**SUMMARY:** Complete loan underwriting analysis has been performed using systematic evaluation of the applicant's financial profile, creditworthiness, and risk factors."""
            
            return AIMessage(content=full_response)
        
        # For other messages, use the SageMaker model normally
        system_msg = """You are a professional loan underwriter. Provide helpful responses about loan underwriting processes."""
        
        full_prompt = f"{{system_msg}}\\n\\nUser: {{user_content}}"
        
        # Get response from SageMaker endpoint
        response = self.sagemaker_llm.invoke(full_prompt)
        
        # Return a proper LangChain AIMessage
        return AIMessage(content=response)

# Define the agent using SageMaker endpoint
def create_agent():
    """Create and configure the LangGraph agent with SageMaker endpoint"""
    
    # Your SageMaker endpoint configuration
    endpoint_name = "{endpoint_name}"
    
    class ContentHandler(LLMContentHandler):
        content_type = "application/json"
        accepts = "application/json"

        def transform_input(self, prompt: str, model_kwargs: Dict) -> bytes:
            # GPT-OSS harmony format payload structure
            payload = {{
                "model": "/opt/ml/model",
                "input": [
                    {{
                        "role": "system",
                        "content": "You are a professional loan underwriter. Analyze loan applications and provide detailed decisions with reasoning."
                    }},
                    {{
                        "role": "user",
                        "content": prompt
                    }}
                ],
                "max_output_tokens": model_kwargs.get("max_new_tokens", 2048),
                "stream": "false",
                "temperature": model_kwargs.get("temperature", 0.1),
                "top_p": model_kwargs.get("top_p", 1)
            }}
            input_str = json.dumps(payload)
            return input_str.encode("utf-8")

        def transform_output(self, output: bytes) -> str:
            # Parse harmony format response
            decoded_output = output.read().decode("utf-8")
            response_json = json.loads(decoded_output)
            
            if 'output' in response_json and isinstance(response_json['output'], list):
                for item in response_json['output']:
                    if item.get('type') == 'message' and item.get('role') == 'assistant':
                        content = item.get('content', [])
                        for content_item in content:
                            if content_item.get('type') == 'output_text':
                                return content_item.get('text', '')
                
                # Fallback parsing for different harmony format structures
                for item in response_json['output']:
                    if item.get('type') != 'reasoning' and 'content' in item and isinstance(item['content'], list):
                        for content_item in item['content']:
                            if content_item.get('type') == 'output_text' and 'text' in content_item:
                                return content_item['text']
                
                for item in response_json['output']:
                    if 'content' in item and isinstance(item['content'], list):
                        for content_item in item['content']:
                            if 'text' in content_item:
                                return content_item['text']
            
            return str(response_json)

    # Initialize SageMaker LLM with harmony format
    content_handler = ContentHandler()
    sagemaker_llm = SagemakerEndpoint(
        endpoint_name=endpoint_name,
        region_name="us-east-2",
        model_kwargs={{
            "max_new_tokens": 2048, 
            "do_sample": True, 
            "temperature": 0.1,  # Lower temperature for consistent underwriting
            "top_p": 1
        }},
        content_handler=content_handler
    )
    
    # Create tools
    tools = [parse_loan_application, analyze_creditworthiness, make_final_decision]
    
    # Wrap SageMaker LLM to work with LangGraph
    llm_with_tools = SagemakerLLMWrapper(sagemaker_llm, tools)
    
    # System message for loan underwriting
    system_message = """You are a professional loan underwriter with expertise in credit analysis and risk assessment. 

Your role is to:
1. Parse loan applications (even conversational ones) to extract key information
2. Analyze creditworthiness based on financial data, credit history, and risk factors
3. Make final approval/denial decisions with detailed reasoning

Use the available tools systematically:
- First, parse the application to extract structured information
- Then, analyze creditworthiness and assess risk factors
- Finally, make a decision with clear reasoning and terms if approved

Be thorough, professional, and provide clear explanations for all decisions."""
    
    # Define the chatbot node
    def chatbot(state: MessagesState):
        # Add system message if not already present
        messages = state["messages"]
        if not messages or not isinstance(messages[0], SystemMessage):
            messages = [SystemMessage(content=system_message)] + messages
        
        response = llm_with_tools.invoke(messages)
        return {{"messages": [response]}}
    
    # Create the graph
    graph_builder = StateGraph(MessagesState)
    
    # Add nodes
    graph_builder.add_node("chatbot", chatbot)
    graph_builder.add_node("tools", ToolNode(tools))
    
    # Add edges
    graph_builder.add_conditional_edges(
        "chatbot",
        tools_condition,
    )
    graph_builder.add_edge("tools", "chatbot")
    
    # Set entry point
    graph_builder.set_entry_point("chatbot")
    
    # Compile the graph
    return graph_builder.compile()

# Initialize the agent
agent = create_agent()

@app.entrypoint
def langgraph_loan_sagemaker(payload):
    """
    Invoke the loan underwriter agent with a payload
    """
    user_input = payload.get("prompt")
    
    # Create the input in the format expected by LangGraph
    response = agent.invoke({{"messages": [HumanMessage(content=user_input)]}})
    
    # Extract the final message content
    return response["messages"][-1].content

if __name__ == "__main__":
    app.run()
'''
    
    # Write the file
    with open(filename, 'w') as f:
        f.write(code_content)
    
    print(f"Created {filename} with endpoint: {endpoint_name}")
    return filename

# Now use it dynamically to create the AgentCore deployment file
create_agentcore_deployment_file(sagemaker_endpoint_name)
print("AgentCore deployment file created successfully!")

## What happens behind the scenes?

When you use `BedrockAgentCoreApp`, it automatically:

* Creates an HTTP server that listens on the port 8080
* Implements the required `/invocations` endpoint for processing the agent's requirements
* Implements the `/ping` endpoint for health checks (very important for asynchronous agents)
* Handles proper content types and response formats
* Manages error handling according to the AWS standards

## Deploying the agent to AgentCore Runtime

The `CreateAgentRuntime` operation supports comprehensive configuration options, letting you specify container images, environment variables and encryption settings. You can also configure protocol settings (HTTP, MCP) and authorization mechanisms to control how your clients communicate with the agent. 

**Note:** Operations best practice is to package code as container and push to ECR using CI/CD pipelines and IaC

In this tutorial can will the Amazon Bedrock AgentCode Python SDK to easily package your artifacts and deploy them to AgentCore runtime.

### Creating IAM Role for Bedrock AgentCore Runtime

Before deploying our loan underwriting agent to Bedrock AgentCore Runtime, we need to create an IAM role with 
the appropriate permissions. This role will allow AgentCore to:

• **Invoke your SageMaker endpoint** for GPT-OSS model inference  
• **Manage ECR repositories** for storing container images  
• **Write CloudWatch logs** for monitoring and debugging  
• **Access Bedrock AgentCore workload services** for runtime operations  
• **Send telemetry data** to X-Ray and CloudWatch for observability  

The function below creates a comprehensive IAM role with five custom policies plus the AWS managed 
AmazonBedrockFullAccess policy. Each policy is scoped to only the resources needed for AgentCore operations, 
following the principle of least privilege.

**Key Features**:  

• **Automatic policy creation** with timestamped names to avoid conflicts  
• **Error handling** for existing roles and policies  
• **Resource-specific permissions** (scoped to your SageMaker endpoint and ECR repositories)  
• **Ready-to-use role ARN** for AgentCore configuration  

This approach ensures your agent has exactly the permissions it needs to run securely in the AgentCore Runtime 
environment while maintaining access to your specific SageMaker model endpoint.


In [ ]:
import boto3
import json
import time
from botocore.exceptions import ClientError

def create_bedrock_agentcore_role(role_name, sagemaker_endpoint_name, region="us-west-2"):
    """
    Create a complete BedrockAgentCore role with all necessary permissions
    
    Args:
        role_name: Name for the IAM role
        sagemaker_endpoint_name: SageMaker endpoint name to grant access to
        region: AWS region (default: us-west-2)
    
    Returns:
        str: Role ARN
    """
    
    # Initialize clients
    iam_client = boto3.client('iam', region_name=region)
    sts_client = boto3.client('sts', region_name=region)
    account_id = sts_client.get_caller_identity()['Account']
    
    print(f"Creating BedrockAgentCore role: {role_name}")
    print(f"Region: {region}, Account: {account_id}")
    print(f"SageMaker Endpoint: {sagemaker_endpoint_name}")
    print("-" * 60)
    
    # 1. Create the IAM Role
    trust_policy = {
        "Version": "2012-10-17",
        "Statement": [{
            "Effect": "Allow",
            "Principal": {"Service": "bedrock-agentcore.amazonaws.com"},
            "Action": "sts:AssumeRole"
        }]
    }
    
    try:
        role_response = iam_client.create_role(
            RoleName=role_name,
            AssumeRolePolicyDocument=json.dumps(trust_policy),
            Description="Custom role for Bedrock AgentCore with SageMaker permissions",
            MaxSessionDuration=3600
        )
        role_arn = role_response['Role']['Arn']
        print(f"Created role: {role_arn}")
    except ClientError as e:
        if e.response['Error']['Code'] == 'EntityAlreadyExists':
            role_arn = f"arn:aws:iam::{account_id}:role/{role_name}"
            print(f"Role already exists: {role_arn}")
        else:
            raise
    
    # 2. Define all required policies
    policies = {
        "SageMakerInvoke": {
            "Version": "2012-10-17",
            "Statement": [{
                "Effect": "Allow",
                "Action": [
                    "sagemaker:InvokeEndpoint",
                    "sagemaker:InvokeEndpointAsync", 
                    "sagemaker:DescribeEndpoint"
                ],
                "Resource": f"arn:aws:sagemaker:us-east-2:{account_id}:endpoint/{sagemaker_endpoint_name}"
            }]
        },
        "ECRManagement": {
            "Version": "2012-10-17",
            "Statement": [{
                "Effect": "Allow",
                "Action": [
                    "ecr:CreateRepository",
                    "ecr:DescribeRepositories", 
                    "ecr:ListTagsForResource",
                    "ecr:TagResource"
                ],
                "Resource": f"arn:aws:ecr:{region}:{account_id}:repository/bedrock-agentcore-*"
            }]
        },
        "ECRAccess": {
            "Version": "2012-10-17",
            "Statement": [
                {
                    "Effect": "Allow",
                    "Action": ["ecr:GetAuthorizationToken"],
                    "Resource": "*"
                },
                {
                    "Effect": "Allow",
                    "Action": [
                        "ecr:BatchGetImage",
                        "ecr:GetDownloadUrlForLayer"
                    ],
                    "Resource": f"arn:aws:ecr:{region}:{account_id}:repository/bedrock-agentcore-*"
                }
            ]
        },
        "Logs": {
            "Version": "2012-10-17",
            "Statement": [{
                "Effect": "Allow",
                "Action": [
                    "logs:CreateLogGroup",
                    "logs:CreateLogStream",
                    "logs:PutLogEvents",
                    "logs:DescribeLogGroups",
                    "logs:DescribeLogStreams"
                ],
                "Resource": f"arn:aws:logs:{region}:{account_id}:log-group:/aws/bedrock-agentcore/runtimes/*"
            }]
        },
        "Workload": {
            "Version": "2012-10-17",
            "Statement": [
                {
                    "Effect": "Allow",
                    "Action": [
                        "bedrock-agentcore:GetWorkloadAccessToken",
                        "bedrock-agentcore:GetWorkloadAccessTokenForJWT",
                        "bedrock-agentcore:GetWorkloadAccessTokenForUserId"
                    ],
                    "Resource": [
                        f"arn:aws:bedrock-agentcore:{region}:{account_id}:workload-identity-directory/default",
                        f"arn:aws:bedrock-agentcore:{region}:{account_id}:workload-identity-directory/default/workload-identity/*"
                    ]
                },
                {
                    "Effect": "Allow",
                    "Action": [
                        "xray:PutTraceSegments",
                        "xray:PutTelemetryRecords",
                        "xray:GetSamplingRules",
                        "xray:GetSamplingTargets"
                    ],
                    "Resource": "*"
                },
                {
                    "Effect": "Allow",
                    "Action": "cloudwatch:PutMetricData",
                    "Resource": "*",
                    "Condition": {
                        "StringEquals": {
                            "cloudwatch:namespace": "bedrock-agentcore"
                        }
                    }
                }
            ]
        }
    }
    
    # 3. Create custom policies
    created_policies = []
    timestamp = str(int(time.time()))
    
    for policy_type, policy_doc in policies.items():
        policy_name = f"BedrockAgentCore{policy_type}Policy-{timestamp}"
        
        try:
            response = iam_client.create_policy(
                PolicyName=policy_name,
                PolicyDocument=json.dumps(policy_doc),
                Description=f"BedrockAgentCore {policy_type} permissions"
            )
            policy_arn = response['Policy']['Arn']
            created_policies.append(policy_arn)
            print(f"Created policy: {policy_name}")
            
        except ClientError as e:
            if e.response['Error']['Code'] == 'EntityAlreadyExists':
                policy_arn = f"arn:aws:iam::{account_id}:policy/{policy_name}"
                created_policies.append(policy_arn)
                print(f"Policy already exists: {policy_name}")
            else:
                print(f"Error creating policy {policy_name}: {e}")
    
    # 4. Attach all policies to the role
    all_policies = created_policies + ["arn:aws:iam::aws:policy/AmazonBedrockFullAccess"]
    
    print("Attaching policies to role...")
    for policy_arn in all_policies:
        try:
            iam_client.attach_role_policy(
                RoleName=role_name,
                PolicyArn=policy_arn
            )
            policy_name = policy_arn.split('/')[-1]
            print(f"Attached policy: {policy_name}")
            
        except ClientError as e:
            if 'already attached' in str(e).lower() or 'EntityAlreadyExists' in str(e):
                policy_name = policy_arn.split('/')[-1]
                print(f"Policy already attached: {policy_name}")
            else:
                print(f"Error attaching policy {policy_arn}: {e}")
    
    # 5. Wait for role to be ready
    print("Waiting for role to be ready...")
    time.sleep(10)
    
    print("-" * 60)
    print(f"SUCCESS: Role created successfully")
    print(f"Role ARN: {role_arn}")
    print(f"Created {len(created_policies)} custom policies")
    
    return role_arn

# Usage example:
# Specify your parameters
role_name = "MyBedrockAgentCoreRole"
region = "us-west-2"

# Create the role
role_arn = create_bedrock_agentcore_role(role_name, sagemaker_endpoint_name, region)

# Use the role ARN in your AgentCore configuration
print(f"\nUse this role ARN in your AgentCore configuration:")
print(f"execution_role='{role_arn}'")

### Configure AgentCore Runtime deployment

First we will use our starter toolkit to configure the AgentCore Runtime deployment with an entrypoint, the execution role we just created and a requirements file. We will also configure the starter kit to auto create the Amazon ECR repository on launch.

During the configure step, your docker file will be generated based on your application code

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
agent_name = "langgraph_loan_parser_agent"

boto_session = Session()
region = "us-west-2"

agentcore_runtime = Runtime()

# Configure the agent (this doesn't require Docker)
response = agentcore_runtime.configure(
    entrypoint="langgraph_loan_sagemaker_gpt_oss.py",
    auto_create_execution_role=False,
    execution_role=role_arn,  # Use custom role
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name,
)

### Launching agent to AgentCore Runtime

Now that we've got a docker file, let's launch the agent to the AgentCore Runtime. This will create the Amazon ECR repository and the AgentCore Runtime

<div style="text-align:left">
    <img src="images/launch.png" width="75%"/>
</div>

In [ ]:
launch_result = agentcore_runtime.launch(use_codebuild=True)

In [ ]:
agent_arn = launch_result.agent_arn
print(f"Agent_ARN='{agent_arn}'")

### Checking for the AgentCore Runtime Status
Now that we've deployed the AgentCore Runtime, let's check for it's deployment status

In [ ]:
import time
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

### Invoking AgentCore Runtime

Finally, we can invoke our AgentCore Runtime with a payload

In [ ]:
invoke_response = agentcore_runtime.invoke({"prompt": "My client is 76 years old. She is nurse making $38,000 per year, been working for 30 years. Needs $22,000 for student loan consolidation. her credit score is 600."})

invoke_response

### Parsing and Displaying Loan Analysis Results

After invoking our loan underwriting agent through AgentCore Runtime, we need to parse and format the response 
for clear presentation. The agent returns a comprehensive analysis in JSON format containing three distinct 
phases of the underwriting process.

This parsing function extracts and displays the loan analysis in a structured format:

Response Processing:
- **Decodes the byte stream** from AgentCore into readable text
- **Parses the JSON response** containing the complete loan analysis
- **Extracts three main sections** using regex pattern matching:
    - Step 1: Application parsing with structured applicant data
    - Step 2: Creditworthiness analysis with financial metrics and risk assessment
    - Step 3: Final decision with approval/denial status and loan terms

Key Information Extraction:  

• **Decision status** (APPROVED/DENIED) with visual indicators  
• **Approval score** showing the quantitative assessment  
• **Loan terms** including interest rate and monthly payment  
• **Applicant summary** with key financial details  

Error Handling:  

• Gracefully handles JSON parsing errors  
• Falls back to plain text display if structured parsing fails  
• Provides debugging information for troubleshooting  

This formatted output makes it easy to review the agent's decision-making process and present professional loan 
underwriting results to stakeholders.

In [ ]:
import json
import re

def parse_bedrock_agentcore_response(invoke_response):
    """Parse the complete Bedrock AgentCore response from byte chunks"""
    
    # Combine all byte chunks into one string
    response_chunks = invoke_response['response']
    complete_response = b''.join(response_chunks).decode('utf-8')
    
    try:
        # Parse the JSON (it's a JSON string containing the analysis)
        data = json.loads(complete_response)
        
        print(" COMPLETE LOAN UNDERWRITING ANALYSIS")
        print("=" * 80)
        
        # Extract the three main sections using regex
        step1_match = re.search(r'\*\*Step 1 - Application Parsing:\*\*(.*?)\*\*Step 2', data, re.DOTALL)
        step2_match = re.search(r'\*\*Step 2 - Creditworthiness Analysis:\*\*(.*?)\*\*Step 3', data, re.DOTALL)
        step3_match = re.search(r'\*\*Step 3 - Final Decision:\*\*(.*?)---', data, re.DOTALL)
        
        print(f"\nSTEP 1 - APPLICATION PARSING:")
        print("-" * 50)
        if step1_match:
            parsed_section = step1_match.group(1).strip()
            print(parsed_section)
        else:
            print("Could not extract parsing section")
        
        print(f"\n STEP 2 - CREDITWORTHINESS ANALYSIS:")
        print("-" * 50)
        if step2_match:
            analysis_section = step2_match.group(1).strip()
            print(analysis_section)
        else:
            print("Could not extract analysis section")
        
        print(f"\n STEP 3 - FINAL DECISION:")
        print("-" * 50)
        if step3_match:
            decision_section = step3_match.group(1).strip()
            print(decision_section)
            
            # Extract key decision info
            decision_match = re.search(r'FINAL LOAN DECISION: ([^\n]+)', decision_section)
            approval_score_match = re.search(r'Approval Score: ([^\n]+)', decision_section)
            monthly_payment_match = re.search(r'Monthly Payment: \$?([\d,]+\.?\d*)', decision_section)
            interest_rate_match = re.search(r'Interest Rate: ([\d.]+)% APR', decision_section)
            
            print(f"\n🎯 KEY DECISION METRICS:")
            print("-" * 30)
            if decision_match:
                status = decision_match.group(1).strip()
                emoji = "✅" if "APPROVED" in status else "❌"
                print(f"{emoji} Decision: {status}")
            if approval_score_match:
                print(f"📈 Approval Score: {approval_score_match.group(1).strip()}")
            if interest_rate_match:
                print(f"💰 Interest Rate: {interest_rate_match.group(1)}% APR")
            if monthly_payment_match:
                print(f"💳 Monthly Payment: ${monthly_payment_match.group(1)}")
        else:
            print("Could not extract decision section")
        
        # Extract applicant summary
        name_match = re.search(r'Name: ([^\n]+)', data)
        income_match = re.search(r'Annual Income: ([^\n]+)', data)
        credit_score_match = re.search(r'Credit Score: ([^\n]+)', data)
        loan_amount_match = re.search(r'Loan Amount Requested: ([^\n]+)', data)
        
        if name_match:
            print(f"\nAPPLICANT SUMMARY:")
            print("-" * 25)
            print(f"Name: {name_match.group(1).strip()}")
            if income_match:
                print(f"Income: {income_match.group(1).strip()}")
            if credit_score_match:
                print(f"Credit Score: {credit_score_match.group(1).strip()}")
            if loan_amount_match:
                print(f"Loan Amount: {loan_amount_match.group(1).strip()}")
        
        print(f"\nANALYSIS COMPLETE")
        print("=" * 80)
        
        return {
            'parsed_application': step1_match.group(1).strip() if step1_match else None,
            'creditworthiness_analysis': step2_match.group(1).strip() if step2_match else None,
            'final_decision': step3_match.group(1).strip() if step3_match else None,
            'raw_response': data
        }
        
    except json.JSONDecodeError as e:
        print(f"JSON Error: {e}")
        print(f"Raw response length: {len(complete_response)}")
        print(f"First 500 chars: {complete_response[:500]}")
        
        # Try to parse as plain text if JSON fails
        print("\n🔍 ATTEMPTING PLAIN TEXT PARSING:")
        print("-" * 40)
        print(complete_response)
        return {'raw_response': complete_response}

# Parse your existing response
loan_analysis = parse_bedrock_agentcore_response(invoke_response)

### Invoking AgentCore Runtime with boto3

Now that your AgentCore Runtime was created you can invoke it with any AWS SDK. For instance, you can use the boto3 `invoke_agent_runtime` method for it.

In [ ]:
import boto3
import json

agent_arn = launch_result.agent_arn
agentcore_client = boto3.client(
    'bedrock-agentcore',
    region_name=region
)

boto3_response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn=agent_arn,
    qualifier="DEFAULT",
    payload=json.dumps({"prompt": "My name is Lisa Chen, 26 years old. I'm a nurse making $68,000 per year, been working for 18 months. Need $22,000 for student loan consolidation. My credit score is 710."})
)

# Use the parsing function we created earlier
loan_analysis = parse_bedrock_agentcore_response(boto3_response)

## Cleanup (Optional)

Let's now clean up the AgentCore Runtime created

In [ ]:
launch_result.ecr_uri, launch_result.agent_id, launch_result.ecr_uri.split('/')[1]

In [ ]:
agentcore_control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=region
)
ecr_client = boto3.client(
    'ecr',
    region_name=region
)
runtime_delete_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id
)

response = ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split('/')[1],
    force=True
)

# Congratulations!